# 1. RAG chain 구현 구문

In [35]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain_community.tools import TavilySearchResults
from langchain_core.documents import Document

# 평가 알로리즘 모듈
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

from ragas import EvaluationDataset, RunConfig, evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from pydantic import BaseModel, Field


# 메모리 관련 모듈
from langchain_core.chat_history import InMemoryChatMessageHistory

from langchain.chains import RetrievalQA, LLMChain
from langchain.schema import AIMessage, HumanMessage




from textwrap import dedent
from operator import itemgetter

from dotenv import load_dotenv
load_dotenv()


True

In [36]:
########################################################
# config 목록
########################################################
COLLECTION_NAME = "bluer_db_openai"
PERSIST_DIRECTORY = "vector_store/chroma/bluer_db"
EMBEDDING_MODEL_NAME = "text-embedding-3-small"
embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)
MODEL_NAME = 'gpt-4o-mini'


########################################################
# vector_db에서 데이터 불러오기
########################################################

# vector store 연결
vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

# 저장된 데이터 내용 확인
documents = vector_store._collection.get()['documents']
metadatas = vector_store._collection.get()['metadatas']

print(f"Documents: {documents[:5]}") 
print(f"Metadatas: {metadatas[:5]}")

Documents: ['id: 4627\nbookStatus: APPROVAL\nfoodTypes: 중식, 일반중식\nheaderInfo_nameKR: 홍보각\nheaderInfo_ribbonType: RIBBON_THREE\ndefaultInfo_phone: 02-531-6479\ndefaultInfo_openHours: nan\ndefaultInfo_closeHours: nan\ndefaultInfo_openHoursWeekend: nan\ndefaultInfo_closeHoursWeekend: nan\ndefaultInfo_dayOff: 연중무휴\ndefaultInfo_app2Yn: True\nstatusInfo_parking: 가능\nstatusInfo_creditCard: y\nstatusInfo_visit: nan\nstatusInfo_menu: 스페셜코스(1인 15만원), 디너코스(A 12만원, B 16만원), 고법불도장(13만원), 모자새우(소 6만원, 대 10만원), 한알탕수육(소 4만8천원, 대 7만원), 특선냉채(2만8천원), 셰프특선수프(3만8천원), 셰프스페셜송이특찜(15만원)\nstatusInfo_priceRange: 10~15만원대\nstatusInfo_businessHours: 12:00~15:00/18:00~21:30(마지막 주문 20:00) \nstatusInfo_newOpenDate: 2007년 \njuso_roadAddrPart1: 서울특별시 강남구 봉은사로 130\njuso_siNm: 서울특별시\njuso_sggNm: 강남구\njuso_rn: 동호로\nreview_review: 국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스토랑. 고급스러운 분위기의 인테리어에서 정통 중식을 맛볼 수 있다. 여경래 셰프의 시그니처 메뉴인 불도장, 모자새우 등을 선보인다.\nreview_reviewSimple: 국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스토랑. 고급스러

In [8]:
print(f"Documents: {documents[:2]}") 
print(f"Metadatas: {metadatas[:2]}")

Documents: ['id: 4627\nbookStatus: APPROVAL\nfoodTypes: 중식, 일반중식\nheaderInfo_nameKR: 홍보각\nheaderInfo_ribbonType: RIBBON_THREE\ndefaultInfo_phone: 02-531-6479\ndefaultInfo_openHours: nan\ndefaultInfo_closeHours: nan\ndefaultInfo_openHoursWeekend: nan\ndefaultInfo_closeHoursWeekend: nan\ndefaultInfo_dayOff: 연중무휴\ndefaultInfo_app2Yn: True\nstatusInfo_parking: 가능\nstatusInfo_creditCard: y\nstatusInfo_visit: nan\nstatusInfo_menu: 스페셜코스(1인 15만원), 디너코스(A 12만원, B 16만원), 고법불도장(13만원), 모자새우(소 6만원, 대 10만원), 한알탕수육(소 4만8천원, 대 7만원), 특선냉채(2만8천원), 셰프특선수프(3만8천원), 셰프스페셜송이특찜(15만원)\nstatusInfo_priceRange: 10~15만원대\nstatusInfo_businessHours: 12:00~15:00/18:00~21:30(마지막 주문 20:00) \nstatusInfo_newOpenDate: 2007년 \njuso_roadAddrPart1: 서울특별시 강남구 봉은사로 130\njuso_siNm: 서울특별시\njuso_sggNm: 강남구\njuso_rn: 동호로\nreview_review: 국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스토랑. 고급스러운 분위기의 인테리어에서 정통 중식을 맛볼 수 있다. 여경래 셰프의 시그니처 메뉴인 불도장, 모자새우 등을 선보인다.\nreview_reviewSimple: 국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스토랑. 고급스러

In [37]:
from langchain_core.output_parsers import StrOutputParser


vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)


# GPT Model 생성
model = ChatOpenAI(
    model=MODEL_NAME,
    temperature=0 
)


# Retriever 생성 - "Map Reduce" 방식 - 더 정확한 답변
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k":5, "fetch_k":10, "lambda_mult":0.5}
)


# Prompt Template 생성
prompt_template = ChatPromptTemplate.from_messages([
    ("system", """당신은 한국의 블루리본 서베이 안내자입니다. 주어진 데이터를 꼼꼼히 확인하여 가능한 정확한 정보를 주세요. 
        반드시 context 의 내용을 바탕으로 대답해 주세요. context에 답변과 관련된 내용이 없으면 모른다고 대답하세요.\n\n[context]\n{context} """),
    ("human", "{question}")
])

# prompt_template = ChatPromptTemplate.from_messages([
#     ("system", "당신은 한국의 블루리본 서베이 전문가입니다. 질문에 자세히 답해주세요."),
#     ("human", "{question}")
# ])


#########################################
# Chain 생성
#########################################

chain =  {'context': retriever, 'question': RunnablePassthrough()}  | prompt_template | model | StrOutputParser()


# retrieval_qa = RetrievalQA.from_chain_type(
#     llm=model,
#     retriever=retriever,
#     return_source_documents=True,  # Include source documents in response
# )
# response = retrieval_qa({"query": QUERY})

# print("응답:", response["result"])



In [38]:
QUERY = "강남구 근처에 음식점 다섯 곳만 추천해주세요"
response = chain.invoke(QUERY)


In [39]:
print(response)

강남구 근처에 추천할 만한 음식점 다섯 곳은 다음과 같습니다:

1. **강남면옥(대치점)**
   - 전화: 02-539-6393
   - 음식 종류: 한식(육류), 소갈비찜, 한식(면류), 함흥냉면
   - 메뉴: 비빔냉면, 회냉면, 물냉면, 섞어냉면(각 8천원), 갈비찜(소 3만5천원, 중 4만5천원, 대 5만5천원)
   - 주소: 서울특별시 강남구 도곡로 428
   - 영업시간: 11:00~22:00, 연중무휴

2. **김영모과자점(강남구청점)**
   - 전화: 02-541-5484
   - 음식 종류: 디저트/베이커리, 베이커리
   - 메뉴: 바게트샌드위치(9천원), 몽블랑(8천원), 연유라우겐(5천원)
   - 주소: 서울특별시 강남구 선릉로126길 6
   - 영업시간: 08:00~22:30, 명절 당일 휴무
   - 웹사이트: [k-bread.com](http://k-bread.com/)

3. **강마을**
   - 전화: 031-555-8828
   - 음식 종류: 한식(민물어패류), 민물매운탕
   - 메뉴: 메기매운탕(대 6만원, 중 5만원, 소 4만원), 장어구이(2마리 8만원)
   - 주소: 경기도 남양주시 강변북로632번길 26
   - 영업시간: 10:00~22:00, 명절 휴무

4. **목포자매집**
   - 전화: 02-543-0729
   - 음식 종류: 한식(일반한식), 남도음식, 한식(어패류)
   - 메뉴: 소낙비(중 8만원, 대 12만원), 민어탕(1인 1만5천원)
   - 주소: 서울특별시 강남구 언주로115길 14
   - 영업시간: 11:30~22:00 (토요일 11:30~21:00), 일요일 및 명절 휴무

5. **마노디셰프(강남점)**
   - 전화: 02-553-5311
   - 음식 종류: 이탈리아식, 피자, 파스타
   - 메뉴: 피자(1만5천6백원~2만3천9백원), 파스타(1만4천8백원~2만7천8백원)
   - 주소: 서울특별시 강남구 테헤란로 129
   - 영업시간: 11:30~24:0

In [40]:
QUERY = "리본이 두개 이상인 식당을 추천해주세요"
response = chain.invoke(QUERY)
print(response)

주어진 데이터에는 리본이 두 개 이상인 식당에 대한 정보가 없습니다. 따라서 추천할 수 있는 식당이 없습니다.


In [34]:
QUERY = " 리본이 있는 레스토랑 중에 서울에 있는 프랑스식 레스토랑 추천해줘"
response = retrieval_qa({"query": QUERY})
print(response)

{'query': ' 리본이 있는 레스토랑 중에 서울에 있는 프랑스식 레스토랑 추천해줘', 'result': '서울에 있는 리본이 있는 프랑스식 레스토랑으로는 다음과 같은 곳이 있습니다:\n\n1. **서촌프렌치**\n   - 전화: 0507-1398-2297\n   - 메뉴: 코스(5만5천원, 7만5천원), 비프브루기뇽(2만9천원), 닭다리살콩피(2만8천원) 등\n   - 가격대: 2~5만원대\n   - 주소: 서울특별시 종로구 옥인길 18\n   - 휴무: 월, 화요일\n\n2. **레썽스**\n   - 전화: 0507-1325-7050\n   - 메뉴: 런치테이스팅코스(6만8천원), 디너테이스팅코스(14만원)\n   - 가격대: 2~5만원대\n   - 주소: 부산광역시 수영구 광남로22번길 17\n   - 휴무: 월, 화요일\n\n이 두 곳은 리본이 있는 프랑스식 레스토랑으로 추천드립니다.', 'source_documents': [Document(metadata={}, page_content='id: 41246\nbookStatus: APPROVAL\nfoodTypes: 프랑스식\nheaderInfo_nameKR: 서촌프렌치\nheaderInfo_ribbonType: RIBBON_ONE\ndefaultInfo_phone: 0507-1398-2297\ndefaultInfo_openHours: nan\ndefaultInfo_closeHours: nan\ndefaultInfo_openHoursWeekend: nan\ndefaultInfo_closeHoursWeekend: nan\ndefaultInfo_dayOff: 월, 화요일 휴무\ndefaultInfo_app2Yn: False\nstatusInfo_parking: 불가\nstatusInfo_creditCard: nan\nstatusInfo_visit: nan\nstatusInfo_menu: 코스(5만5천원, 7만5천원), 비프브루기뇽(2만9천), 닭다리살콩피(2만8천원), 파리지앵뇨키, 라구파스타, 바질페스

In [32]:
print(response)

{'query': ' 예약 불가능한 레스토랑 추천해줘', 'result': '예약이 불가능한 레스토랑으로는 "리브나"가 있습니다. 이곳은 월요일에 휴무이며, 주류 주문이 필수인 다이닝바입니다.', 'source_documents': [Document(metadata={}, page_content='id: 32587\nbookStatus: APPROVAL\nfoodTypes: 중식, 모던차이니즈, 홍콩식중식, 북경오리\nheaderInfo_nameKR: 팔레드신\nheaderInfo_ribbonType: RIBBON_TWO\ndefaultInfo_phone: 051-922-5100  \ndefaultInfo_openHours: nan\ndefaultInfo_closeHours: nan\ndefaultInfo_openHoursWeekend: nan\ndefaultInfo_closeHoursWeekend: nan\ndefaultInfo_dayOff: 연중무휴 \ndefaultInfo_app2Yn: False\nstatusInfo_parking: 가능\nstatusInfo_creditCard: y\nstatusInfo_visit: 해운대역 3번 출구에서 해운대 해수욕장 방면으로 600미터\nstatusInfo_menu: 런치코스(7만5천원~13만5천원), 디너코스(12만5천원~24만5천원), 북경오리(15만원), 소흥주칠리새우, 훈연향메로(각 6만원), 이베리코차슈, 동파육(6만원), 삼선짜장면(2만6천원), 삼선짬뽕(3만2천원) \nstatusInfo_priceRange: 5~10만원대\nstatusInfo_businessHours: 12:00~14:30/18:00~21:30 | 토, 일요일 12:00~15:00/17:30~21:30\nstatusInfo_newOpenDate: 2020년 10월 \njuso_roadAddrPart1: 부산광역시 해운대구 해운대해변로 292\njuso_siNm: 부산광역시\njuso_sggNm: 해운대구\njuso_rn: nan\nr

## 기록

```python
# # Retriever 생성 - 성능이 안좋음: 부정확한 답변
# retriever = vector_store.as_retriever(
#     search_type="similarity",
#     search_kwargs={"k": 5}, 
# )

```

## 데이터 검토를 위한 코드들

In [15]:
# Retriever를 통해 관련 문서 가져오기
documents = retriever.get_relevant_documents(QUERY)

# 관련 문서 출력
for idx, doc in enumerate(documents):
    print(f"문서 {idx + 1}:")
    print("Metadata:", doc.metadata)
    print("Content:", doc.page_content)
    print("------")

문서 1:
Metadata: {}
Content: id: 4091
createdDate: 1216620773000
bookStatus: INACTIVE
chefName: nan
closeHours: nan
closeHoursWeekend: nan
dayOff: 명절 휴무
openHours: nan
openHoursWeekend: nan
phone: 02-307-9979
website: nan
gps_latitude: 37.57189
gps_longitude: 126.934353
ribbonType: nan
juso_detailAddress: nan
juso_roadAddrPart1: 서울특별시 서대문구 연희로25길 31-3
review_text: 가정집을 개조한 아늑한 분위기의 한정식집. 떡갈비와 간장게장이 인기 메뉴며 반찬도 정갈하다. 가정집에 초대받아 음식을 대접받는 기분으로 식사를 즐길 수 있다. 가격 대비 만족도도 높은 편이다.
review_simple: 가정집을 개조한 아늑한 분위기의 한정식집. 떡갈비와 간장게장이 인기 메뉴며 반찬도 정갈하다. 가정집에 초대받아 음식을 대접받는 기분으로 식사를 즐길 수 있다. 가격 대비 만족도도 높은 편이다.
statusInfo_businessHours: 11:30~15:00/17:00~21:00 
statusInfo_creditCard: y
statusInfo_menu: 떡갈비정식(2만1천원), 간장게장정식(3만8천원), 보리굴비정식(2만6천원), 수빈정식세트(4인 7만8천원~10만9천원)
foodType_1: 한식(일반한식)
foodDetailType_1: 한정식
foodType_2: 한정식
foodDetailType_2: 게장
foodType_3: 한식(어패류)
foodDetailType_3: nan
foodType_4: 게장
foodDetailType_4: nan
foodType_5: nan
foodDetailType_5: nan
foodType_6: nan
foodDetailType_6: nan
foodTyp